In [1]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy

# basic module
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import shutil
import glob
import time
import pickle

plt.style.use('seaborn-dark')
%config InlineBackend.figure_format = 'retina'
pd.options.display.max_rows = 20
pd.options.display.max_columns = 20

plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.color'] = 'r'
plt.rcParams['axes.grid'] = False


# image module
import cv2
# import pylibjpeg
from PIL import Image



# dataset
# df = pd.read_csv('dataset.csv',index_col=0)

import warnings

warnings.filterwarnings('ignore')

In [ ]:
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import Dropout

pre_model = MobileNetV3Small(weights='imagenet', input_shape=(224, 224, 3),include_top=False)

x = GlobalAveragePooling2D()(pre_model.output)
output = Dense(units=7, activation='softmax', name='output_layer')(x)
mobilenet = Model(inputs=pre_model.input, outputs=output)

6709248/6698480 [==============================] - 0s 0us/step


In [ ]:
mobilenet.summary()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/3차플젝drive/공모전/dataset_.csv')

In [4]:
data.head()

,Unnamed: 0,identifier,sex,age_group,age,height,weight,report,pneumonia_type,orginal_dcm_file,orginal_png_file,crop_file,body_part_file,diagnosis
0,0,H05_00019_02,M,A1,0개월,NaN,0.9,NaN,NaN,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/crop/d4/H05_00019_02.png,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,4
1,1,H01_00716_01,F,A1,0개월,52.0,3.9,NaN,NaN,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/crop/d4/H01_00716_01.png,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,4
2,2,H03_00172_01,F,A1,0개월,42.0,1.8,NaN,NaN,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/crop/d4/H03_00172_01.png,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,4
3,3,H05_00204_01,M,A1,0개월,50.5,3.0,NaN,NaN,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/crop/d4/H05_00204_01.png,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,4
4,4,H04_00178_01,F,A1,0개월,30.0,0.5,Diffuse hazziness in both lungs. ETT insertion...,NaN,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,/home/ncp/workspace/data/crop/d4/H04_00178_01.png,/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A...,4


In [5]:
org_list = data['crop_file']
crop_list = []

for i in org_list:
    chg = i.replace('/home/ncp/workspace/data/', '/content/drive/MyDrive/3차플젝drive/공모전/')
    crop_list.append(chg)

In [ ]:
plt.imread(crop_list[0]).shape

(534, 707, 4)

In [ ]:
def cvt_crop(og_img_path, size):
    
    og_img = cv2.imread(og_img_path)
    
    
    # clahe
    clahe = cv2.createCLAHE(5.0,(8,8))
    
    ## og_img_convert
    cvt_img = cv2.resize(og_img, size, interpolation = cv2.INTER_AREA)
    cvt_img = cv2.cvtColor(cvt_img,cv2.COLOR_BGR2GRAY)
    cvt_img = clahe.apply(cvt_img)  # 1 channel
    cvt_img = np.repeat(cvt_img[:,:,np.newaxis],3,-1)

    return cvt_img

In [ ]:
from tqdm import tqdm

X = np.ndarray((4000,224,224,3))

for i in tqdm(range(len(X))):
    X[i][:][:][:] = cvt_crop(crop_list[i], (224,224))

100%|██████████| 4000/4000 [1:27:26<00:00,  1.31s/it]


In [ ]:
y = np.array(data['diagnosis'])

In [ ]:
from sklearn.model_selection import train_test_split

Xtr, Xva, Ytr, Yva = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)
Xva, Xtst, Yva, Ytst = train_test_split(Xva,Yva, test_size=0.5, random_state=42, stratify=Yva)

In [ ]:
from tensorflow.keras.utils import to_categorical

Ytr_c = to_categorical(Ytr)
Yva_c = to_categorical(Yva)
Ytst_c = to_categorical(Ytst)

In [ ]:
print(Xtr.shape ,'/', Ytr.shape ,'/', Ytr_c.shape)
print(Xva.shape ,'/', Yva.shape ,'/', Yva_c.shape)
print(Xtst.shape ,'/', Ytst.shape ,'/', Ytst_c.shape)

(3200, 224, 224, 3) / (3200,) / (3200, 7)
(400, 224, 224, 3) / (400,) / (400, 7)
(400, 224, 224, 3) / (400,) / (400, 7)


In [ ]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator


#data augmentation 

# Create two instances (images and targets) with the same arguments
data_gen_args = dict(horizontal_flip=True, vertical_flip = True, rotation_range = 10)
image_datagen = ImageDataGenerator(**data_gen_args)
# mask_datagen = ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1

train_generator = image_datagen.flow(x=Xtr, y=Ytr_c,
                                    batch_size=32,
                                    shuffle=True)
validation_generator = image_datagen.flow(x=Xva,y=Yva_c,
                                         batch_size=32,
                                         shuffle=True)


In [ ]:
from tensorflow.keras import backend as K
import tensorflow.python as tf
from sklearn.metrics import classification_report



def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision

def f1_score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score


def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)



class_weight = {0:0.571428563,
                1:1.397990367,
                2:4.617604545,
                3:1.662337636,
                4:0.892857129,
                5:0.571428563,
                6:1.181247674}

from tensorflow.keras.optimizers import Adam
b

mobilenet.compile( 
    optimizer= Adam(lr=1e-6), 
    loss='categorical_crossentropy', 
#     metrics=[tf.keras.metrics.CategoricalAccuracy()])
    metrics=['accuracy',recall,precision,f1_score])

In [ ]:
from tensorflow.keras.application import DenseNet121

pre_trained_model = DenseNet121(input_shape = (128, 128, 3), 
                                include_top = False, 
                                weights = 'imagenet')

for layer in pre_trained_model.layers:
  layer.trainable = False

predictions = tf.keras.layers.Dense(14, activation='sigmoid', name='predictions')(pre_trained_model.output)

pre_trained_model = tf.keras.Model(inputs=pre_trained_model.input, outputs=predictions)

pre_trained_model.load_weights('/content/drive/MyDrive/3차플젝drive/KAGGLE_UNET/CheXNet_Keras_0.3.0_weights.h5')

x = GlobalAveragePooling2D()(pre_trained_model.layers[-3].output)
output = Dense(units=7, activation='softmax', name='output_layer')(x)
chexnet_model = Model(inputs=pre_trained_model.input, outputs=output)

chexnet_model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy',f1_score])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


es = EarlyStopping( monitor='val_f1score',
                min_delta=0, 
                patience=0, 
                verbose=0, 
                mode='auto', 
                baseline=None, 
                restore_best_weights=False )

checkpointer = ModelCheckpoint('mobilenet_ver1.hdf5', monitor='val_f1_score' ,
                               mode='max', verbose=1,save_best_only=True ,save_weights_only=False)

In [ ]:
# chexnet_model.compile(optimizer='nadam', loss='binary_crossentropy',metrics=['acc'])
import tensorflow as tf

# history = chexnet_model.fit(Xtr,Ytr_c, validation_data=[Xva,Yva_c],batch_size=3, epochs=100, callbacks=[checkpointer])
history2 = mobilenet.fit_generator(train_generator, validation_data=validation_generator, 
                                       epochs=150, steps_per_epoch = 3, class_weight = class_weight,
                                       callbacks=[checkpointer],verbose=1)

In [ ]:
history = mobilenet.fit(Xtr, Ytr_c, validation_data=[Xva,Yva_c], batch_size=64 ,epochs = 250)

Epoch 1/250
50/50 [==============================] - 12s 178ms/step - loss: 1.9429 - accuracy: 0.2016 - recall: 0.0000e+00 - precision: 0.0000e+00 - f1_score: 0.0000e+00 - val_loss: 2.0422 - val_accuracy: 0.1975 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_f1_score: 0.0000e+00
Epoch 2/250
50/50 [==============================] - 9s 177ms/step - loss: 1.9145 - accuracy: 0.2216 - recall: 0.0000e+00 - precision: 0.0000e+00 - f1_score: 0.0000e+00 - val_loss: 2.0382 - val_accuracy: 0.1775 - val_recall: 0.0048 - val_precision: 0.0385 - val_f1_score: 0.0085
Epoch 3/250
50/50 [==============================] - 9s 177ms/step - loss: 1.8889 - accuracy: 0.2303 - recall: 0.0000e+00 - precision: 0.0000e+00 - f1_score: 0.0000e+00 - val_loss: 2.0104 - val_accuracy: 0.2175 - val_recall: 0.0072 - val_precision: 0.0897 - val_f1_score: 0.0133
Epoch 4/250
50/50 [==============================] - 9s 178ms/step - loss: 1.8658 - accuracy: 0.2478 - recall: 0.0000e+00 - precision: 0.0000e+00 - f1

In [ ]:
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

resnet = ResNet50(include_top=True, weights=None, input_shape=(224, 224, 3), pooling=max, classes=7)
out = resnet.output
out2 = Dense(7, activation='sigmoid')(out)
c_model = Model(resnet.input, out2)

In [ ]:
c_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc',precision, recall, f1_score])

history = c_model.fit_generator(train_generator, epochs=100, steps_per_epoch = 5, validation_data=validation_generator)

In [ ]:
history = c_model.fit(Xtr, Ytr_c, validation_data=[Xva,Yva_c], batch_size=3 ,epochs = 150)